In [1]:
import re
import string
import numpy as np
import random
import os

# os.environ['TF_NUM_INTEROP_THREADS'] = '4'
# os.environ['TF_NUM_INTRAOP_THREADS'] = '4'

import tensorflow as tf

In [2]:
with open('1984.txt') as f:
    base1984 = f.read()

In [3]:
print(base1984[:1000])

'test'

1984 



By George Orwell 




i 



iiwy"^ 



Part One 



1984 



Chapter i 



It was a bright cold day in April, and the clocks were strik- 
ing thirteen. Winston Smith, his chin nuzzled into his 
breast in an effort to escape the vile wind, slipped quickly 
through the glass doors of Victory Mansions, though not 
quickly enough to prevent a swirl of gritty dust from enter- 
ing along with him. 

The hallway smelt of boiled cabbage and old rag mats. At 
one end of it a coloured poster, too large for indoor display, 
had been tacked to the wall. It depicted simply an enor- 
mous face, more than a metre wide: the face of a man of 
about forty- five, with a heavy black moustache and rugged- 
ly handsome features. Winston made for the stairs. It was 
no use trying the lift. Even at the best of times it was sel- 
dom working, and at present the electric current was cut 
off during daylight hours. It was part of the economy drive 
in preparation for Hate Week. The flat was seve

In [4]:
# print("^[{}\s]+$".format(re.escape(''.join(filter(lambda x: x != "'", string.punctuation)))))
filterd1984 = re.split(r'!|(?<!\b[MmDd][Rr])\.|\?|;', base1984)
# isPunct = re.compile("^[{}\s]+$".format(re.escape(string.punctuation)))
isPunctNoAp = re.compile("[{}§—„]+".format(re.escape(''.join(filter(lambda x: x != "'" and x != '.', string.punctuation)))))
filterd1984 = list(map(lambda s: ' '.join(filter(lambda x: not re.match(r'^\s*$', x) and not len(x) == 0,
    re.sub(r'\s+', ' ',
    isPunctNoAp.sub(' ',
    re.sub(r'(?<!\b)\'\b|\b\'(?!\b)', ' ',
    re.sub("-\s*\n", "",
    s.lower())))).split())), filterd1984))
print(filterd1984[:5])

['test 1984 by george orwell i iiwy part one 1984 chapter i it was a bright cold day in april and the clocks were striking thirteen', 'winston smith his chin nuzzled into his breast in an effort to escape the vile wind slipped quickly through the glass doors of victory mansions though not quickly enough to prevent a swirl of gritty dust from entering along with him', 'the hallway smelt of boiled cabbage and old rag mats', 'at one end of it a coloured poster too large for indoor display had been tacked to the wall', 'it depicted simply an enormous face more than a metre wide the face of a man of about forty five with a heavy black moustache and ruggedly handsome features']


In [5]:
print("\n".join(filterd1984))

test 1984 by george orwell i iiwy part one 1984 chapter i it was a bright cold day in april and the clocks were striking thirteen
winston smith his chin nuzzled into his breast in an effort to escape the vile wind slipped quickly through the glass doors of victory mansions though not quickly enough to prevent a swirl of gritty dust from entering along with him
the hallway smelt of boiled cabbage and old rag mats
at one end of it a coloured poster too large for indoor display had been tacked to the wall
it depicted simply an enormous face more than a metre wide the face of a man of about forty five with a heavy black moustache and ruggedly handsome features
winston made for the stairs
it was no use trying the lift
even at the best of times it was seldom working and at present the electric current was cut off during daylight hours
it was part of the economy drive in preparation for hate week
the flat was seven flights up and winston who was thirty nine and had a varicose ulcer above his 

In [6]:
n = 16
mark = {}
for sent in filterd1984:
    for i in range(0, len(sent)+1):
        prev = sent[max(0, i-n):i]
        tt = mark.get(prev, '')
        if i == len(sent):
            mark[prev] = tt + '!'
        else:
            mark[prev] = tt + sent[i]

In [7]:
def finishSentMark(s):
    # assert len(s) >= n
    while True:
        crand = mark.get(s[-n:], '!')
        c = random.choice(crand)
        if c == '!':
            return s
        s += c

In [29]:
print(finishSentMark(''))

now the scary nights returned and the mental terror the holo cell had demanded of him


## "expected":

> The cops were armed 
with shotguns and tied Frank, who almost slipped on the 
ground, and also the other inmate, with an additional seat 
belt

## predicted:

> with shotguns and the policeman would finally you surrendered but he could not hold his stool out at arm's length of the dead seemed to strike down large crowds

## tailed questions

> for the moment is merely a piece of evidence of your sister isn't it

In [25]:
# print(finishSentMark(''))
print(finishSentMark('brig'))

brig


In [10]:
letters = '!' + ''.join(sorted(set(''.join(filterd1984))))
print(letters)
print(len(letters))

! '.0123456789abcdefghijklmnopqrstuvwxyz
40


In [11]:
from functools import reduce

"""
xy = \
    [ ( w[max(0, i-n):i], (w)[i:] ) # + '!'
    for w in filterd1984
    for i in range(len(w)+1)
    ]
"""

desiredlen = 100

# filterd1984_1 = list(filter(lambda x: len(x) <= desiredlen, filterd1984))
filterd1984_1 = list(map(lambda x: re.sub('\s+', '', x[:desiredlen]), filterd1984))
print(len(filterd1984_1))

maxlen = reduce(max, map(len, filterd1984_1))

xy = \
    [ ( w[:-1], w[1:] )
    for w in filterd1984_1
    ]

12189


In [12]:
xy[300]

('com23moredangerousthantheinitialactofopeningthediarybutforamomenthewastemptedto',
 'om23moredangerousthantheinitialactofopeningthediarybutforamomenthewastemptedtot')

In [13]:
def recode(x):
    ggg = np.array([letters.index(c) for c in x])
    return np.append(ggg, np.zeros(maxlen - len(ggg)))

def recodey(y):
    ggg = np.array([letters.index(c) for c in y])
    a =  np.append(ggg, np.zeros(maxlen - len(ggg), dtype=int))
    b = np.zeros((a.size, len(letters)), dtype=int)
    try:
        b[np.arange(a.size, dtype=int), a] = 1
    except Exception as e:
        return b
    return b

xy_train = xy

xy1_x = np.array(
    [ recode(x).reshape(maxlen,1)
    for x, y in xy_train
    ])

xy1_y = np.array(
    [ recodey(y) #.reshape(maxlen, len(letters))
    for x, y in xy_train
    ])

In [14]:
print(xy1_y[0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [15]:
lstm = tf.keras.layers.LSTM(len(letters), return_sequences=True, activation='softmax')
model = tf.keras.models.Sequential(
    [ lstm
    # , tf.keras.layers.Dense(len(letters), activation='softmax')
     # a bit better with ^
    ])
print("x\t", xy1_x[0].shape)
print('out\t', lstm(xy1_x[0].reshape(1,*xy1_x[0].shape)).shape)
print("y\t", xy1_y[0].shape)
print("len\t", len(xy1_x))
print("sentlen\t", maxlen)

x	 (89, 1)
out	 (1, 89, 40)
y	 (89, 40)
len	 12189
sentlen	 89


In [16]:
# print(model.predict(xy1_x[0].reshape(1,*xy1_x[0].shape)))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer='sgd', # adagrad
              metrics=['accuracy'])
model.fit(xy1_x, xy1_y, epochs=3);
# print(model.predict(xy1_x[0].reshape(1,*xy1_x[0].shape)))

Epoch 1/3
136/381 [=========>....................] - ETA: 10s - loss: 3.3485 - accuracy: 0.3966

KeyboardInterrupt: 

In [ ]:
xy1_y

In [ ]:
def runLSTM(x):
    print('from ', x)
    print(len(x))
    if len(x) > 0 and x[-1] == '!' or len(x) >= maxlen:
        return x
    x0 = x
    x = recode(x)
    x = x.reshape(1,*x.shape)
    p = model.predict(x)
    # print(p)
    p = np.argmax(p, axis=2)[0]
    print('#', ''.join(map(lambda x: letters[x], p)), sep='')
    p = p[len(x0)-1:]
    # p = np.round(p)
    # return x0 + ''.join(map(lambda x: letters[x], p))
    return runLSTM(x0 + letters[p[0]])
runLSTM('e')

In [ ]:
runLSTM('q')

In [ ]:
runLSTM('bright col')